In [4]:
!pip install boto3

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [ ]:
! pip install --upgrade sagemaker

In [7]:
!pip install boto3
### Permissions and environment variables
import re
import boto3
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
region = boto3.Session().region_name

# S3 bucket donde ya están los datasets públicos de SageMaker
downloaded_data_bucket = f"sagemaker-example-files-prod-{region}"
downloaded_data_prefix = "datasets/image/MNIST"

# Usa el bucket de SageMaker ya creado en tu cuenta (en vez de sess.default_bucket())
bucket = f"sagemaker-{region}-196289957086"
prefix = "sagemaker/DEMO-linear-mnist"

# Define IAM role
role = "arn:aws:iam::196289957086:role/service-role/c174660a4519461l11376159t1w1-SageMakerExecutionRole-3lRQvl6oPPuN"

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'boto3'

In [ ]:
### Data ingestion

import urllib.request
import gzip
import pickle

url = "http://deeplearning.net/data/mnist/mnist.pkl.gz"
urllib.request.urlretrieve(url, "mnist.pkl.gz")

with gzip.open("mnist.pkl.gz", "rb") as f:
    train_set, valid_set, test_set = pickle.load(f, encoding="latin1")

In [ ]:
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker

sagemaker_session = sagemaker.Session()

# Cargar datos directamente a S3 (si tu laboratorio lo permite)
from sagemaker.amazon.common import write_numpy_to_dense_tensor
import numpy as np
import io

# Ejemplo con datos dummy para ilustrar
train_data = np.random.rand(100, 784)
train_labels = np.random.randint(10, size=(100,))
buf = io.BytesIO()
write_numpy_to_dense_tensor(buf, train_data.astype('float32'), train_labels.astype('float32'))
buf.seek(0)

bucket = f"sagemaker-{region}-196289957086"
prefix = "sagemaker/DEMO-linear-mnist"
key = f"{prefix}/train_data"
boto3.client("s3").upload_fileobj(buf, bucket, key)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (2, 10)


def show_digit(img, caption="", subplot=None):
    if subplot is None:
        _, (subplot) = plt.subplots(1, 1)
    imgr = img.reshape((28, 28))
    subplot.axis("off")
    subplot.imshow(imgr, cmap="gray")
    plt.title(caption)


show_digit(train_set[0][30], f"This is a {train_set[1][30]}")

In [ ]:
import io
import numpy as np
import sagemaker.amazon.common as smac

train_set_vectors = np.array([t.tolist() for t in train_set[0]]).astype("float32")
train_set_labels = np.where(np.array([t.tolist() for t in train_set[1]]) == 0, 1, 0).astype(
    "float32"
)

validation_set_vectors = np.array([t.tolist() for t in valid_set[0]]).astype("float32")
validation_set_labels = np.where(np.array([t.tolist() for t in valid_set[1]]) == 0, 1, 0).astype(
    "float32"
)

train_set_buf = io.BytesIO()
validation_set_buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(train_set_buf, train_set_vectors, train_set_labels)
smac.write_numpy_to_dense_tensor(validation_set_buf, validation_set_vectors, validation_set_labels)
train_set_buf.seek(0)
validation_set_buf.seek(0)